## Dynamic Flow

### Phase-I

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from langchain_ollama import ChatOllama

In [2]:
from typing import TypedDict, List, Optional, Annotated
from langgraph.graph import StateGraph, END
import operator

class Task(TypedDict):
    id: str
    description: str
    assigned_agent: str 
    status: str

class AgentState(TypedDict):
    requirements: str
    project_root: str

    architecture: Optional[str]
    execution_plan: Optional[str]
    task_queue: List[Task]

    current_task: Optional[Task]
    recent_code_changes: List[str]

    test_logs: Optional[str]
    error_analysis: Optional[str]

    iteration_count: int  

### Creating Tools

In [3]:
import os
import subprocess
from langchain_core.tools import tool



@tool
def write_file(file_path: str, content: str):
    """
    Writes content to a file. 
    Args:
        file_path: strict path under ./builds/app-x/
        content: the full code or text to write
    """

    if "builds/" not in file_path:
        return "Error: You can only write inside the 'builds/' directory."
    
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "w") as f:
            f.write(content)
        return f"Successfully wrote {len(content)} bytes to {file_path}"
    except Exception as e:
        return f"Error writing file: {e}"

@tool
def read_file(file_path: str):
    """Reads a file. Args: file_path"""
    try:
        with open(file_path, "r") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"



@tool
def run_shell_command(command: str, work_dir: str):
    """
    Executes a terminal command.
    Args:
        command: e.g., 'python main.py' or 'npm install'
        work_dir: the directory to run in (e.g., ./builds/app-1)
    """
    try:
        result = subprocess.run(
            command,
            cwd=work_dir,
            shell=True,
            capture_output=True,
            text=True,
            timeout=20
        )
        if result.returncode == 0:
            return f"SUCCESS:\n{result.stdout}"
        else:
            return f"FAILED:\nSTDOUT: {result.stdout}\nSTDERR: {result.stderr}"
    except Exception as e:
        return f"System Error: {e}"

llm = ChatOllama(
    model="llama3.1:8b", 
    temperature=0,
    format="json" 
)

llm_worker = llm.bind_tools([write_file, read_file, run_shell_command])

### Creating Agent Phase-II

In [4]:
import json
from prompts import *
import operator

def architect_node(state: AgentState):
    print(f"\n🏗️  Architecting {state['project_root']}...")
    msg = architect_prompt.format(
        project_root=state["project_root"],
        requirements=state["requirements"]
    )
    response = llm.invoke(msg)
    return {"architecture": response.content}


def orchestrator_node(state: AgentState):
    print("\n🎼 Orchestrating...")
    msg = orchestrator_prompt.format(
        architecture=state["architecture"]
    )
    response = llm.invoke(msg)
    return {"execution_plan": response.content}


def planner_node(state: AgentState):
    print("\n📅 Planning Tasks...")

    # --------------------------------------------------
    # 1. DO NOT REPLAN IF TASKS ALREADY EXIST
    # --------------------------------------------------
    if state.get("task_queue"):
        print("⏭️  Planner skipped: existing tasks in queue")
        return {}

    errs = state.get("error_analysis")

    msg = planner_prompt.format(
        execution_plan=state.get("execution_plan"),
        error_analysis=errs,
        project_root=state["project_root"]
    )

    response = llm.invoke(msg)

    try:
        content = response.content

        # --------------------------------------------------
        # 2. STRIP MARKDOWN FENCES
        # --------------------------------------------------
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0]
        elif "```" in content:
            content = content.split("```")[1].split("```")[0]

        parsed = json.loads(content)

        # --------------------------------------------------
        # 3. UNWRAP COMMON SHAPES
        # --------------------------------------------------
        if isinstance(parsed, dict) and "tasks" in parsed:
            parsed = parsed["tasks"]

        if isinstance(parsed, dict):
            parsed = [parsed]

        if not isinstance(parsed, list):
            raise ValueError("Planner output must be a list of tasks")

        # --------------------------------------------------
        # 4. NORMALIZE + ENFORCE TASK SCHEMA
        # --------------------------------------------------
        new_tasks = []

        for task in parsed:
            if not isinstance(task, dict):
                raise ValueError(f"Invalid task type: {type(task)}")

            description = task.get("description")
            if not description:
                raise ValueError(f"Task missing description: {task}")

            assigned_agent = task.get("assigned_agent", "backend")

            # Planner NEVER executes tasks
            if assigned_agent == "planner":
                assigned_agent = "backend"

            normalized_task = {
                "id": task.get("id", f"task_{len(new_tasks) + 1}"),
                "description": description,
                "assigned_agent": assigned_agent,
                "status": task.get("status", "pending"),
            }

            new_tasks.append(normalized_task)

        print(f"✅ Generated {len(new_tasks)} tasks")

        return {
            "task_queue": new_tasks,
            "error_analysis": None,
            "iteration_count": state.get("iteration_count", 0) + 1
        }

    except Exception as e:
        print(f"❌ Planner JSON Error: {e}")
        return {
            "task_queue": [],
            "error_analysis": str(e)
        }



def worker_node(state: AgentState):
    if not state["task_queue"]:
        return {}

    current_task = state["task_queue"][0]
    remaining_tasks = state["task_queue"][1:]

    # ---- DEFENSIVE CHECK (saves your sanity) ----
    if not isinstance(current_task, dict):
        raise TypeError(
            f"Invalid task format. Expected dict, got {type(current_task)}: {current_task}"
        )

    print(
        f"\n👷 {current_task['assigned_agent'].upper()} Working on: "
        f"{current_task['description']}"
    )

    msg = backend_prompt_template.format(
        task_description=current_task["description"],
        project_root=state["project_root"]
    )

    llm_worker.invoke(msg)

    return {
        "current_task": current_task,
        "task_queue": remaining_tasks,
        "recent_code_changes": [
            f"Executed task: {current_task['description']}"
        ]
    }


def tester_node(state: AgentState):
    print("\n🧪 Testing...")
    msg = tester_prompt_template.format(
        project_root=state["project_root"]
    )
    response = llm_worker.invoke(msg)
    return {"test_logs": response.content}


def reviewer_node(state: AgentState):
    print("\n🧐 Reviewing Results...")
    logs = state.get("test_logs") or ""

    if "SUCCESS" in logs and "Error" not in logs:
        return {"error_analysis": None}

    msg = reviewer_prompt.format(test_logs=logs)
    response = llm.invoke(msg)
    return {"error_analysis": response.content}


def check_queue(state: AgentState):
    if state["task_queue"]:
        return "work"
    return "test"


def check_review(state: AgentState):
    if state.get("error_analysis") is None:
        return "end"
    if state.get("iteration_count", 0) >= 5:
        print("❌ Max retries reached.")
        return "end"
    return "fix"


workflow = StateGraph(
    AgentState,
    reducers={
        "task_queue": operator.add,
        "recent_code_changes": operator.add,
    }
)

workflow.add_node("architect", architect_node)
workflow.add_node("orchestrator", orchestrator_node)
workflow.add_node("planner", planner_node)
workflow.add_node("worker", worker_node)
workflow.add_node("tester", tester_node)
workflow.add_node("reviewer", reviewer_node)

workflow.set_entry_point("architect")
workflow.add_edge("architect", "orchestrator")
workflow.add_edge("orchestrator", "planner")

workflow.add_conditional_edges(
    "planner",
    check_queue,
    {"work": "worker", "test": "tester"}
)

workflow.add_edge("worker", "planner")
workflow.add_edge("tester", "reviewer")

workflow.add_conditional_edges(
    "reviewer",
    check_review,
    {"fix": "planner", "end": END}
)

app = workflow.compile()


In [14]:
inputs = {
    "requirements": "Create a simple Python calculator CLI that logs history to a text file.",
    "project_root": "./builds/app-calculator",

    "architecture": None,
    "execution_plan": None,
    "task_queue": [],

    "current_task": None,
    "recent_code_changes": [],

    "test_logs": None,
    "error_analysis": None,

    "iteration_count": 0
}


# Run
for output in app.stream(inputs):
    # This prints the state updates as they happen
    for key, value in output.items():
        print(f"Finished: {key}")


🏗️  Architecting ./builds/app-calculator...
Finished: architect

🎼 Orchestrating...
Finished: orchestrator

📅 Planning Tasks...
✅ Generated 3 tasks
Finished: planner

👷 BACKEND Working on: Install required dependencies and set up the project structure
Finished: worker

📅 Planning Tasks...
⏭️  Planner skipped: existing tasks in queue
Finished: planner

👷 BACKEND Working on: Write Python code for the calculator application in `calculator.py` file
Finished: worker

📅 Planning Tasks...
⏭️  Planner skipped: existing tasks in queue
Finished: planner

👷 BACKEND Working on: Modify the `history.txt` file to log user's input and output in JSON format
Finished: worker

📅 Planning Tasks...
✅ Generated 3 tasks
Finished: planner

👷 BACKEND Working on: Install required dependencies and set up the project structure
Finished: worker

📅 Planning Tasks...
⏭️  Planner skipped: existing tasks in queue
Finished: planner

👷 BACKEND Working on: Write Python code for the calculator application in `calculator.

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT